In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
excel_path = 'Data_all.xlsx'  # Replace with the actual path to your Excel file
data = pd.read_excel(excel_path)
print(data.head())
print(data['stu_answer'])
data.dropna(subset=['stu_answer'])
data['stu_answer'].isnull().sum()

answers = data['stu_answer'].values
grades = data['grade'].values

question = data['id_question'].values
answers = [str(answer) for answer in answers]
tokenizer = Tokenizer(filters=''''!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''''' )
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(answers)
print(tokenizer.word_index)
vocab_size = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(answers)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')
print(padded_sequences.shape)
sequences = tokenizer.texts_to_sequences(answers)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')
print(padded_sequences.shape)
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, grades, test_size=0.2, random_state=42)

   id_question                             stu_answer  grade
0          1.0                                  رمضان      2
1          1.0                              شهر رمضان      2
2          1.0                              شهر الصوم      0
3          1.0              الشهر الذي نزل فيه القرآن      1
4          1.0  شهر العبادة والتقرب إلى الله هو رمضان      2
0                                                 رمضان
1                                             شهر رمضان
2                                             شهر الصوم
3                             الشهر الذي نزل فيه القرآن
4                 شهر العبادة والتقرب إلى الله هو رمضان
                             ...                       
1416       طالت دعوة الرسول صلى الله عليه وسلم 23 عاماً
1417      استغرقت دعوة الرسول صلى الله عليه وسلم 23 سنة
1418    استغرقت دعوة الرسول صلى الله عليه وسلم 23 عاماً
1419                                   استغرقت 23 عاماً
1420                                     استغرقت 23 سنة
Name: stu_answer, 

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW,BertTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

# Load the dataset
excel_path = 'Data_all.xlsx'  # Replace with the actual path to your Excel file
data = pd.read_excel(excel_path)
data.dropna(subset=['stu_answer','grade'], inplace=True)  # Drop rows with missing answers

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("asafaya/bert-base-arabic")
data['stu_answer'] = data['stu_answer'].str.replace(r"[\'\"\!\#\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~\t\n\`÷×؛<>_()\*\&\^%\]\[ـ،\/\:\؟\.\,'{}\~¦\+\|!”…“–ـ]", "")

# Tokenize the answers
tokenized_answers = tokenizer(data['stu_answer'].tolist(), padding=True, truncation=True, return_tensors='pt', max_length=64)

# Labels (grades)

# Combine tokenized answers and labels into a PyTorch dataset
dataset = TensorDataset(tokenized_answers['input_ids'], tokenized_answers['attention_mask'])


C:\Users\AzComputer\AppData\Local\Temp\ipykernel_23332\1372423941.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  data['stu_answer'] = data['stu_answer'].str.replace(r"[\'\"\!\#\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~\t\n\`÷×؛<>_()\*\&\^%\]\[ـ،\/\:\؟\.\,'{}\~¦\+\|!”…“–ـ]", "")


In [4]:
from transformers import BertForMaskedLM, AdamW, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import torch
import pandas as pd

# Load the dataset
excel_path = 'Data_all.xlsx'  # Replace with the actual path to your Excel file
data = pd.read_excel(excel_path)
data.dropna(subset=['stu_answer'], inplace=True)  # Drop rows with missing answers

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("asafaya/bert-base-arabic")

# Tokenize the answers
tokenized_answers = tokenizer(data['stu_answer'].tolist(), padding=True, truncation=True, return_tensors='pt', max_length=64)

# Mask some tokens in the input data to create labels
labels = tokenized_answers['input_ids'].clone()
probability_matrix = torch.full(labels.shape, 0.15)
mask_arr = torch.bernoulli(probability_matrix).bool()
labels[~mask_arr] = -100  # We only compute loss on masked tokens

# Combine tokenized answers and labels into a PyTorch dataset
dataset = TensorDataset(tokenized_answers['input_ids'], tokenized_answers['attention_mask'], labels)
train_size = len(dataset)

# Create data loaders
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Load the pre-trained Arabic model for sequence classification
model = BertForMaskedLM.from_pretrained("asafaya/bert-base-arabic")

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.train()

for epoch in range(num_epochs):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': batch[2].to(device)}

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\AzComputer\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 3/3: 100%|████

`BertForMaskedLM` is a good choice for certain types of assistance for students, particularly in tasks related to language understanding and generation. 

Masked Language Modeling (MLM), the task `BertForMaskedLM` is designed for, involves predicting a masked token in a sentence. This can be useful for tasks like:

- **Autocompletion**: The model can predict what word might come next in a sentence, which can be used to build an autocompletion feature.
- **Grammar and style suggestions**: By predicting what word should be in a masked spot, the model can potentially identify grammar or style issues in the original text.
- **Paraphrasing and text generation**: The model can be used to generate new text or paraphrase existing text, which can be useful for helping students understand a concept in different words.

However, the effectiveness of `BertForMaskedLM` (or any model) will depend on the specific task and how the model is trained and used. For example, if you're trying to provide assistance on a task that isn't related to language (like math homework), `BertForMaskedLM` might not be the best choice. 

Also, keep in mind that while `BertForMaskedLM` can generate predictions for masked tokens, interpreting these predictions and turning them into useful feedback for students is a separate challenge that will require additional logic and possibly additional models.

In [5]:
from transformers import pipeline, BertLMHeadModel

# Load the fine-tuned model for sequence classification
model = BertLMHeadModel.from_pretrained("ha cla",is_decoder=True)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Example text generation for a student answer
example_answer = " في التواصل"
generated_text = generator(example_answer, max_length=18, num_return_sequences=1)[0]['generated_text']

print("Original Answer:")
print(example_answer)
print("\nGenerated Assistance:")
print(generated_text)


Some weights of BertLMHeadModel were not initialized from the model checkpoint at ha cla and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original Answer:
 في التواصل

Generated Assistance:
 في التواصل الهندار المسافر الوص الوص المحكم الخمس الوص الرباط وقعفيت الساح 0راض الاداريديدا
